In [8]:
# Необходимо собрать информацию о вакансиях на вводимую должность 
# (используем input или через аргументы получаем должность) 
# с сайтов HH(обязательно) и/или Superjob(по желанию). 
# Приложение должно анализировать несколько страниц сайта 
# (также вводим через input или аргументы). 
# Получившийся список должен содержать в себе минимум:
# Наименование вакансии.
# Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
# Ссылку на саму вакансию.
# Сайт, откуда собрана вакансия.
# По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). 
# Структура должна быть одинаковая для вакансий с обоих сайтов. 
# Общий результат можно вывести с помощью dataFrame через pandas. 
# Сохраните в json либо csv.

In [9]:
!pip install pymongo

In [2]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import json 
import re
from pymongo import MongoClient 
from pymongo.errors import DuplicateKeyError as dke

In [6]:
url = 'https://api.hh.ru/vacancies'
req = requests.get(url)
req.status_code

200

In [11]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient['vacancies_db']
hh_vacansies = db.hh_vacansies

vacancy_input = input('Введите наименование вакансии: ')
vacancies = []
url = 'https://api.hh.ru/vacancies'
page = 0
escape = False
while url:
    page_next = None
    response = None
    soup = None
    params={'area': '1',
             'st': 'searchVacancy',
             'text': vacancy_input,
             'page': page}
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'}
    response = requests.get(
        "https://api.hh.ru/vacancies", 
        params=params,
        headers=headers
    )
    
    json_response = response.json()
    repository = json_response['items'][0]
       
    pprint(page_next)
    soup = bs(response.text, 'html.parser')
    vacancy_list = soup.find_all('div', attrs='vacancy-serp-item')
    page_next = soup.find('a', attrs={'class': 'bloko-button', 'data-qa': 'pager-next'})
    pprint(page_next)
    page_current = soup.find('span', attrs={'class': 'bloko-button bloko-button_pressed', 'data-qa': 'pager-page'})
    for vacancy in vacancy_list:
        max_vacancy_salary = ''
        vacancy_data = {}
        vacancy_info = vacancy.find('a', attrs={'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'})
        if vacancy_info is not None:
            vacancy_name = vacancy_info.text
            vacancy_herf = vacancy_info.get('href')
            id = ''.join(re.findall(r'\d{8}', vacancy_herf))
            vacancy_info_company = vacancy.find('div', attrs={'class': 'vacancy-serp-item__meta-info-company'})
            if vacancy_info_company is not None:
                vacancy_company = vacancy_info_company.text
                vacancy_company = vacancy_company.replace(u'\xa0', u'')
                if vacancy_company == 'Meta Sistem':
                    vacancy_company = None
                vacancy_info_salary = vacancy.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})
                if vacancy_info_salary is not None:
                    vacancy_salary = vacancy_info_salary.getText().replace(u'\xa0', u'')
                    vacancy_salary = re.split('\s|<|>', vacancy_salary)
                    max_vacancy_salary = ''
                    min_vacancy_salary = ''
                    general_salary = ''
                    if vacancy_salary[0] == 'до':
                        for elements in range(1, len(vacancy_salary) - 1):
                            max_vacancy_salary = max_vacancy_salary + ''.join(vacancy_salary[elements])
                        max_vacancy_salary = int(max_vacancy_salary)
                        min_vacancy_salary = None
                    elif vacancy_salary[0] == 'от':
                        for elements in range(1, len(vacancy_salary) - 1):
                            min_vacancy_salary = min_vacancy_salary + ''.join(vacancy_salary[elements])
                        min_vacancy_salary = int(min_vacancy_salary)
                        max_vacancy_salary = None
                    elif vacancy_salary[0] == 'по':
                        min_vacancy_salary = None
                        max_vacancy_salary = None
                    else:
                        for elements in range(len(vacancy_salary) - 1):
                            general_salary = general_salary + ''.join(vacancy_salary[elements])
                        general_salary = re.split(r'-', general_salary)
                        min_vacancy_salary = int(general_salary[0])
                        max_vacancy_salary = int(general_salary[1])
                    vacancy_data['_id'] = id
                    vacancy_data['Vacancy_name'] = vacancy_name
                    vacancy_data['Vacancy_company'] = vacancy_company
                    vacancy_data['Vacancy_salary_max'] = max_vacancy_salary
                    vacancy_data['Vacancy_salary_min'] = min_vacancy_salary
                    try:
                        hh_vacancies.insert_one(vacancy_data)
                    except dke:
                        pprint('Document already exists')
                page += 1
                pprint(f'Search page: {page}')
                if page_next is None:
                    break
                        
            check_search = input('Search? y/n')
            if check_search == 'y':
                insert_salary = int(input('Insert min salary:'))
                for doc in hh_vacancies.find(
                        {'$or': [{'Vacansy_salary_max': {'$gt': insert_salary}}, {'min_vacancy_salary': {'$gt': insert_salary}}]}):
                    pprint(doc)

Введите наименование вакансии: manager
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 